### Import Libraries

In [37]:
import pandas as pd
import numpy as np
import os
from datetime import datetime 
from dateutil import relativedelta
import os

# Required for airflow
from data_access.dremio_access import DremioAccess
from data_access.s3_access import S3Access
from sns.sns_publish import SNSClient
from data_access.smtp_mailer import SMTPMailer

# Instantiate connection objects for user use
s3_access = S3Access()
sns_client = SNSClient()
email = SMTPMailer()

dremio_access = DremioAccess()

INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting new HTTP connection (1): 169.254.169.254
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting new HTTP connection (1): 169.254.169.254


Please enter your AD password to connect to Dremio: ··············


### Intialise Dates Variables

In [71]:
def date_param(rollback=0):
    #YYYYMM DAtes
    date_param.this_month = (datetime.today() - relativedelta.relativedelta(months=0+rollback)).strftime('%Y%m')
    date_param.last_month = (datetime.today() - relativedelta.relativedelta(months=1+rollback)).strftime('%Y%m')
    date_param.two_monthago = (datetime.today() - relativedelta.relativedelta(months=2+rollback)).strftime('%Y%m')
    
    date_param.This_Month_6MBack_YYYYMM = (datetime.today() - relativedelta.relativedelta(months=6+rollback)).strftime('%Y%m')
    date_param.This_Month_12MBack_YYYYMM = (datetime.today() - relativedelta.relativedelta(months=12+rollback)).strftime('%Y%m')
    
    #YYYYMMDD Dates
    #date_param.This_Month_Start = (date_param.this_month- relativedelta.relativedelta(months=0)).strftime('%Y-%m-%d')
    #date_param.Last_Month_Start = (date_param.last_month- relativedelta.relativedelta(months=0)).strftime('%Y-%m-%d')
    
    date_param.This_Month_Start = datetime.strptime(str(date_param.this_month),'%Y%m').date()
    date_param.Last_Month_Start = datetime.strptime(str(date_param.last_month),'%Y%m').date()

    date_param.This_Month_12MBack_Start = datetime.strptime(str(date_param.This_Month_12MBack_YYYYMM),'%Y%m').date()
    
    date_param.This_Month_Plus_3MBack = (date_param.This_Month_Start - relativedelta.relativedelta(months=3)).strftime('%Y-%m-%d')
    date_param.This_Month_Plus_6MBack = (date_param.This_Month_Start - relativedelta.relativedelta(months=6)).strftime('%Y-%m-%d')
    date_param.This_Month_Plus_3MForward = (date_param.This_Month_Start + relativedelta.relativedelta(months=3)).strftime('%Y-%m-%d')
    date_param.This_Month_10MBack = (date_param.This_Month_Start - relativedelta.relativedelta(months=11)).strftime('%Y-%m-%d')
    date_param.This_Month_75YBack = (date_param.This_Month_Start - relativedelta.relativedelta(years=75)).strftime('%Y-%m-%d')
    
    date_param.Last_Month_MonthEnd = (date_param.Last_Month_Start + pd.offsets.BMonthEnd()).strftime('%Y-%m-%d')

    
    print('This_Month ->', date_param.this_month)
    print('Last_Month ->', date_param.last_month)
    print('Two_MonthAgo ->', date_param.two_monthago)
    
    print('This_Month_Start ->',date_param.This_Month_Start)
    print('Last_Month_Start ->',date_param.Last_Month_Start)
    print('This_Month_12MBack_Start ->',date_param.This_Month_12MBack_Start)
    
    
    print('date_param.This_Month_10MBack ->', date_param.This_Month_10MBack)
    print('date_param.This_Month_Plus_3MBack ->', date_param.This_Month_Plus_3MBack)
    print('date_param.This_Month_Plus_6MBack ->', date_param.This_Month_Plus_6MBack)
    print('date_param.This_Month_Plus_3MForward ->', date_param.This_Month_Plus_3MForward)
    
    print('date_param.This_Month_75YBack ->', date_param.This_Month_75YBack)
    print('date_param.Last_Month_MonthEnd ->', date_param.Last_Month_MonthEnd)
    
    print('date_param.This_Month_6MBack ->', date_param.This_Month_6MBack_YYYYMM)
    print('date_param.This_Month_12MBack ->', date_param.This_Month_12MBack_YYYYMM)


### Check Dates

In [72]:
date_param(5)

This_Month -> 202109
Last_Month -> 202108
Two_MonthAgo -> 202107
This_Month_Start -> 2021-09-01
Last_Month_Start -> 2021-08-01
This_Month_12MBack_Start -> 2020-09-01
date_param.This_Month_10MBack -> 2020-10-01
date_param.This_Month_Plus_3MBack -> 2021-06-01
date_param.This_Month_Plus_6MBack -> 2021-03-01
date_param.This_Month_Plus_3MForward -> 2021-12-01
date_param.This_Month_75YBack -> 1946-09-01
date_param.Last_Month_MonthEnd -> 2021-08-31
date_param.This_Month_6MBack -> 202103
date_param.This_Month_12MBack -> 202009


### Intialise FolderName Variable

In [5]:
FolderName = 'Debenhams September Backdate'  
FolderName

'Debenhams September Backdate'

### Initial Cycle Level Population
#### Creates a data set with core population of accounts and relevant FV fields
- Customer Identification (Account/Customer/Universal Number)
- Customer Status (Balance, Blocks, Status)
- Customer Management Score (User Field 43/User Score 33)
- Age > 75 Years Calculated
- Interest Charge Flag (For IBB)
- Inflight RBP 
- Last RBP Direction 
- CLI Offer Last 1M

#### Creates Inital FV Exclusions
- Blockcode
- Blockcode L3M
- Over 75 Years Old
- Less Than 12 Months On Book
- Card Expired
- Fin Diff
- Debt Management Company (User Amount 13)

In [94]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS commercial_s3.cobrands.Arun.RBP."{FolderName}".Inital_Accounts""")

dremio_access.execute_sql_ctas(
f""" create table commercial_s3.cobrands.Arun.RBP."{FolderName}".Inital_Accounts as 
select
A.account_number
,A.customer_number 
,a.universal_account_identifier
,A.record_date -- this is the record date at month end
,substring(A.account_number,4,3) as org
,substring(A.account_number,7,3) as logo
,b.current_balance
,case when a.interest_net > 0 then 1 else 0 end as interest_charge_flag
,A.credit_limit
,A.account_status_code
,b.last_delinquent_date
,b.card_expiration_date
,a.state_of_issue
,b.pct_control_id
,b.block_code_1
,b.block_code_2
,b.block_code_1_date
,b.block_code_2_date
,b.opened_date
,case when b.opened_date >= '2015-08-01' then 1 
        else 0 end as post_aug_2015_flag
,b.co_maker_date_of_birth
,c.date_of_birth
,cast(substring(cast(date_of_birth as varchar),1,6) as int) as dob_year_month
,case when to_date(c.date_of_birth) < '{date_param.This_Month_75YBack}' then 1 else 0 end as over_75
,b.re_price_date_last_re_priced
,b.re_price_effective_date
,b.user_amount_13 
,b.user_field_43
,d.user_score_31
,d.user_field_44

,case when substr(b.SOURCE_CODE,16,5) = '99950' then 'POS'
      when substr(b.SOURCE_CODE,13,1) = 'D' then 'Internet'
        else 'POS' end as marketing_channel_on_app
        
       
,a.triad_re_pricing_reason
,case when a.triad_re_pricing_reason in ('01', '02' ) and b.re_price_effective_date <= '{date_param.This_Month_Plus_3MForward}' 
    and b.re_price_effective_date > '{date_param.This_Month_Start}' then 1 else 0 end as inflight_reprice
    
,case when  b.re_price_date_last_re_priced >= '{date_param.This_Month_10MBack}' and a.triad_re_pricing_reason = '03' then 'DOWN' 
        when  b.re_price_date_last_re_priced >= '{date_param.This_Month_10MBack}' and a.triad_re_pricing_reason = '04' then 'UP' end as last_reprice_direction

,case when TIMESTAMPDIFF(MONTH,date_trunc('month', b.triad_date_credit_limit_offer), date_trunc('month', a.record_date)) <=1 then 1 else 0 end as CLI_offer_l1m

,case when b.block_code_1 is not null then 1
        when b.block_code_2 is not null then 1
        else 0 end as blocked
        
,case when b.block_code_1_date >= '{date_param.This_Month_Plus_3MBack}' then 1
        when b.block_code_2_date >='{date_param.This_Month_Plus_3MBack}' then 1 else 0 end as blocked_l3m
        
,case when b.last_delinquent_date >= '{date_param.This_Month_Plus_3MBack}' then 1
        else 0 end as del_l3m        
        
,case when b.pct_control_id is null then a.state_of_issue
        when trim(b.pct_control_id) = '' then a.state_of_issue
        else b.pct_control_id end as pct
        
,case when TIMESTAMPDIFF(MONTH,date_trunc('month', b.opened_date), date_trunc('month', a.record_date)) < 12 then 1 
        else 0 end as less_12_mob
        
,case when b.card_expiration_date <= date_add(a.record_date,67)  then 1 
        else 0 end as card_expired
        
,case when b.USER_AMOUNT_13>0 THEN 1
        ELSE 0 END AS DMC     
        
,case when b.user_field_43 = 1 then 1
        else 0 end as fin_diff_flag

,case when TIMESTAMPDIFF(MONTH,date_trunc('month', b.re_price_date_last_re_priced), date_trunc('month', a.record_date)) < 3 then 1 else 0 end as repriced_l6m
        
,case when substring(A.account_number,4,3) = 730 then 'Debenhams'
      when substring(A.account_number,4,3) = 731 then 'House of Fraser'
      when substring(A.account_number,4,3) = 732 then 'Arcadia'
      when substring(A.account_number,4,3) = 733 then 'Laura Ashley'
      when substring(A.account_number,4,3) = 736 THEN 'Amazon'
      else 'Check' end as client
      
,e.class
,e.client_brand_description
,e.brand_description



-- Updated From Cycle
FROM edm.dm_cb_account_monthly_summary a

inner join "edm_pii"."dm_cb_account_daily_snapshot" b
on a.account_number = b.account_number
and b.record_date = a.record_date
and b.record_date between cast('{date_param.This_Month_Plus_3MBack}' as date) and cast('{date_param.Last_Month_MonthEnd}' as date)

inner join "edl_pii"."dl_cb_fd_nameadd" c 
on a.customer_number = c.customer_number
and a.record_date = c.record_date
and c.record_date between '{date_param.This_Month_Plus_3MBack}' and '{date_param.Last_Month_MonthEnd}'

inner join "edm_pii"."dm_cb_account_daily_snapshot" D -- just for score, join on last day of month
on a.account_number = d.account_number
and d.record_Date = '{date_param.Last_Month_MonthEnd}'

left join edm.product_dim e
on substring(A.account_number,4,3) = e.brand_code
and substring(A.account_number,7,3) = e.product_code

where a.record_year_month = '{date_param.last_month}'

and (trim(b.spid) in ('1','2','5','91','92')
  OR substring(A.account_number,4,3) = 736
    )
and b.test_digits not in (20,21,22,23,24,25,26,27,28,29)
and a.account_status_code not in ('T') --  debt transferred
and b.closed_date is null

and b.charge_off_date is null


""", show_metrics = True)                             

   Rows Written  Bytes Written Run Time           Start Time  \
0       1766355      123473354  0:02:59  2022-02-10 15:31:06   

              End Time  
0  2022-02-10 15:34:05  


### Persistent Debt Data Pull For Later Exclusions
• Populates 18, 36 and PD Cure Flags

In [97]:
dremio_access.execute_sql(f"""DROP TABLEl IF EXISTS commercial_s3.cobrands.Arun.RBP."{FolderName}".pd18 """)

dremio_access.execute_sql_ctas(
f""" create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".pd18
as select
account_number as account_number
,record_date
,max(eighteen_month_persistent_debt_flag) as pd_18_flag
,max(_36_month_persistent_debt_flag) as pd_36_flag
,max(_36_month_cure_date) as pd_36_cure_date
from edl."dl_cb_fd_persistent_debt"
where record_date = '{date_param.Last_Month_MonthEnd}'

group by 1,2
""", show_metrics = True)
;

   Rows Written  Bytes Written Run Time           Start Time  \
0       3632657       33073198  0:00:03  2022-02-10 15:43:28   

              End Time  
0  2022-02-10 15:43:31  


''

### PD + Bureau Data Exclusions (TransUnion and Experian) 
- PD Flags Pulled Through
- CCJ L6M (How is this data source reflective of 6 Months?)
- 3+ Searches L1M 
- Mortgage Arrears 
- 2+ On An  

In [100]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS commercial_s3.cobrands.Arun.RBP."{FolderName}".Initial_PD18 """)

dremio_access.execute_sql_ctas(
f""" create table commercial_s3.cobrands.Arun.RBP."{FolderName}".Initial_PD18
as 
select
distinct 
a.*
,cast(to_char(a.record_date,'YYYYMM') as int) as record_yyyymm
,b.pd_18_flag
,b.pd_36_cure_date
,c.nd
,case when c.nd between 0 and 6 then 1 else 0 end as ccj_l6m
,c.ygc
,case when ygc >=3 then 1 else 0 end as searches_3_plus_l1m
,c.gbc
,case when c.gbc in ('1','2','3','4','5','6','D') then 1 else 0 end as mortgage_arrears

,d.e1_b_08_sp
,case when trim(d.e1_b_08_sp) in ('2', '3', '4', '5', '6', '7', '8', '9') then 1 else 0 end as external_two_plus

from "commercial_s3".cobrands.Arun.RBP."{FolderName}".Inital_Accounts a

left join "commercial_s3".cobrands.Arun.RBP."{FolderName}".pd18 b
on a.account_number = b.account_number

left join edl.dl_cb_bureau_tu_bsb c
on a.account_number = c.account_number
and c.record_year_month  = cast(to_char(a.record_date,'YYYYMM') as int)

left join edl.dl_cb_bureau_exp_dcm d
on a.customer_number = d.customer_number
and d.record_year_month = cast(to_char(a.record_date,'YYYYMM') as int)

""", show_metrics = True)
;

   Rows Written  Bytes Written Run Time           Start Time  \
0       1766355      153069345  0:03:43  2022-02-10 15:46:13   

              End Time  
0  2022-02-10 15:49:56  


''

### Account Activity L3M 
#### Selects All Of The FINAL Working Day Of Each Month From DIM Table
- Joins/Filters These Days On the Account Daily Snapshot Table (PII)
- Checks to see if Last Statement Balance (At Month End) Is >0 

In [101]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS commercial_s3.cobrands.Arun.RBP."{FolderName}".activity_l3m""")

dremio_access.execute_sql_ctas(
f"""create table commercial_s3.cobrands.Arun.RBP."{FolderName}".activity_l3m
as

select
a.account_number
,sum(case when b.last_statement_balance > 0 then 1 else 0 end) as months_active
from "commercial_s3".cobrands.Arun.RBP."{FolderName}".Initial_PD18 a
left join edm_pii.dm_cb_account_daily_snapshot  b
on a.account_number = b.account_number
and b.record_date between '{date_param.This_Month_Plus_3MBack}' and '{date_param.This_Month_Start}'
WHERE B.record_date in (select distinct last_working_day_of_month 
                                                from edm.date_dim  
                                                where full_date >= '{date_param.This_Month_Plus_3MBack}'
                        )
GROUP BY a.account_number

""", show_metrics = True);


   Rows Written  Bytes Written Run Time           Start Time  \
0       1766355       19999419  0:14:05  2022-02-10 15:50:53   

              End Time  
0  2022-02-10 16:04:58  


### Credit Limit Change (CLI or CLD?) L6M 
- Takes the past 6 (Will Have To Run A Single Account Through?) Cycles from the table
- Outputs The Credit_Limit_Date When It's More Than 2 Months Away From Opened Date?

In [103]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".activity_l6m""")

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".activity_l6m
as

select a.account_number
    ,a.record_date
    ,case when timestampdiff(month, date_trunc('month', b.opened_date), 
                                    date_trunc('month', credit_limit_date)
                                    ) > 2 then credit_limit_date 
          ELSE NULL END as credit_limit_date
    from "commercial_s3".cobrands.Arun.RBP."{FolderName}".Initial_PD18 a

    left join edm."dm_cb_account_cycle_end_summary" b
    --edm_pii.dm_cb_account_daily_snapshot  b
    on a.account_number = b.account_number

    and b.record_year_month between '{date_param.This_Month_6MBack_YYYYMM}' and '{date_param.last_month}'
    
    
""", show_metrics = True);


   Rows Written  Bytes Written Run Time           Start Time  \
0      10000526      111473580  0:00:04  2022-02-10 16:08:47   

              End Time  
0  2022-02-10 16:08:51  


In [104]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".activity_l6m_2 """)
dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".activity_l6m_2 
as
select
account_number
,record_date
,max(credit_limit_date) as credit_limit_date
from "commercial_s3".cobrands.Arun.RBP."{FolderName}".activity_l6m

group by 1,2
""", show_metrics = True);


   Rows Written  Bytes Written Run Time           Start Time  \
0       1766355       20579793  0:00:04  2022-02-10 16:14:28   

              End Time  
0  2022-02-10 16:14:32  


### BT or MT Promo Balance
- Checks if customer has BT/MT Promo Balance On Date Of Record (ME)

In [106]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".btmt_promo""")

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".btmt_promo
as

select a.account_number
,a.record_date
,max(case when c.promo_0_percent_flag  = '1' and b.current_balance > 0 then 1 else 0 end) as bt_mt_balance

from "commercial_s3".cobrands.Arun.RBP."{FolderName}".Initial_PD18 a

inner join edm.dm_cb_account_plan_snapshot  b
on a.account_number = b.account_number
and b.record_date = a.record_date
--and plan_number in (60006,60009,60012,60106,60109,60112,60118,60124,80006,80009,80010,80015) 
--replaced with join to plan lookup

inner JOIN edm.dr_cb_plan_category_dim c
    ON b.plan_number = c.plan_number
    and b.brand = c.brand
    and c.plan_category in ('BT', 'MT')


group by 1,2

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0         74914        1269740  0:04:50  2022-02-10 16:18:55   

              End Time  
0  2022-02-10 16:23:45  


### Historic Activity Exclusions
- CLI L5M (3M Ago Is Tracked Because Reprice Period is 60 Days + Current Date)
- BT/MT Balance (At Current Month End)
- No Account Activity In Last 3M 

In [107]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".exclusions""")

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".exclusions
as 
select 
a.*
,b.months_active 
,c.credit_limit_date
,d.bt_mt_balance

,case when credit_limit_date IS NULL THEN 0
    WHEN a.record_date > date_add(credit_limit_date,90) then 0 else 1 end as cli_l5m
-- we do 3 months ago as there is still 2 months implementation
        
,case when d.bt_mt_balance > 0 then 1 
        else 0 end as bt_mt_balance_flag
        
from "commercial_s3".cobrands.Arun.RBP."{FolderName}".Initial_PD18 a

left join "commercial_s3".cobrands.Arun.RBP."{FolderName}".activity_l3m b
on a.account_number = b.account_number

left join "commercial_s3".cobrands.Arun.RBP."{FolderName}".activity_l6m_2 c
on a.account_number = c.account_number

left join "commercial_s3".cobrands.Arun.RBP."{FolderName}".btmt_promo d
on a.account_number = d.account_number


""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0       1766355      158457628  0:00:28  2022-02-10 16:23:45   

              End Time  
0  2022-02-10 16:24:13  


In [23]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS  "commercial_s3".cobrands.Arun.RBP."{FolderName}".exclusions_product """)

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".exclusions_product as
select 
a.account_number
,a.user_score_31
,user_field_44
,a.pct
,a.blocked
,a.blocked_l3m
,a.card_expired
,a.del_l3m
,a.dmc
,a.external_two_plus
,a.mortgage_arrears
,a.ccj_l6m
,a.searches_3_plus_l1m
,a.over_75
,a.less_12_mob
,a.months_active
,a.bt_mt_balance_flag
,a.cli_l5m
,CLI_offer_l1m
,a.pd_18_flag
,pd_36_cure_date
,a.fin_diff_flag
,a.repriced_l6m
,a.inflight_reprice
,b.class
,b.client_brand_description
,b.brand_description
,a.marketing_channel_on_app
,post_aug_2015_flag
,a.record_date
,a.opened_date
,current_balance
,credit_limit
,last_reprice_direction
,interest_charge_flag

from "commercial_s3".cobrands.Arun.RBP."{FolderName}".exclusions a

left join edm.product_dim b
on a.org = brand_code
and a.logo = b.product_code
""", show_metrics = True);


   Rows Written  Bytes Written Run Time           Start Time  \
0       1766355       77222940  0:00:02  2022-02-11 09:38:12   

              End Time  
0  2022-02-11 09:38:14  


### Pulling Current APR From PCT Master List
- List is static and was manually generated, ensure completeness before use
- Expectation is that we would lose accounts from the table as invalid PCTs are excluded

In [24]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".current_apr""")

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".current_apr
as select 
a.*
,b.retail_apr
,b.bt_apr
,b.cash_apr
from "commercial_s3".cobrands.Arun.RBP."{FolderName}".exclusions_product a
left join "commercial_s3".cobrands.Perry."RBP_PCT_LIST".PCT_list b
on cast(trim(a.pct) as int) = cast(b.pct as int) 
where substring(a.pct,1,1) in ('0','1','2','3','4','5','6','7','8','9')

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0       1711471       84623706  0:00:03  2022-02-11 09:38:22   

              End Time  
0  2022-02-11 09:38:25  


### Affordability Step 1) Pulling Disposable Income
#### We use the month prior for affordability as it is uploaded via batch and most likely after cycle.
- Disposable Income Comes From Credit Risk
- Best View Pull For Later User
- Aggregate across a number of fields 
- SoleTurn tracks
- JoinTurn tracks
- Metrics tracked over at 'Best View' and over a 12 month period at 3M intervals

In [25]:
def make_case_statement(var_array):
    lrg_str = ""
    for var in var_array:
        small_str = f"""
       case when {var}_900 = 'G' then 900
       when {var}_800 = 'G' then 800
       when {var}_700 = 'G' then 700
       when {var}_600 = 'G' then 600
       when {var}_550 = 'G' then 550
       when {var}_500 = 'G' then 500
       when {var}_450 = 'G' then 450
       when {var}_400 = 'G' then 400
       when {var}_350 = 'G' then 350
       when {var}_300 = 'G' then 300
       when {var}_250 = 'G' then 250
       when {var}_200 = 'G' then 200
       when {var}_175 = 'G' then 175
       when {var}_150 = 'G' then 150
       when {var}_125 = 'G' then 125
       when {var}_100 = 'G' then 100
       when {var}_75 = 'G' then 75
       when {var}_50 = 'G' then 50
       when {var}_25 = 'G' then 25
       when {var}_0 = 'G' then 0
       when {var}_0 = 'A' then -1
       when {var}_0 = 'R' then -2
       when {var}_0 = 'X' then -3
       end as Max_{var} 
       """
        if var == var_array[-1]:
            lrg_str += small_str
            break
        else:
            lrg_str += small_str + ','
    return lrg_str

In [26]:
variables = ["RAG_BEST_VIEW", "RAG_SOLETURN_3M", "RAG_SOLETURN_6M", "RAG_SOLETURN_12M", "RAG_JOINTTURN_3M", "RAG_JOINTTURN_6M", "RAG_JOINTTURN_12M"]
case_str = make_case_statement(variables)

In [27]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".affordability """)

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".affordability 
as select 
ACCNO AS ACCOUNT_NUMBER
,ACCOUNT_LIMIT
,ACCOUNT_BALANCE
,CASE WHEN ICF_BESTVIEW IS NULL THEN 0 ELSE ICF_BESTVIEW END AS ICF_BESTVIEW
,{case_str}
,AFFORD_SCORE1


from edl.dl_cb_tu_affordability a

left join edl.dl_cb_ds_affordability_scores b
ON A.ACCNO = B.ACCOUNT_NUMBER
and cast(to_char(b.RECORD_DATE,'YYYYMM') as int) = {date_param.last_month}


where RECORD_YEAR_MONTH = '{date_param.last_month}'

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0       3204209       88495487  0:00:37  2022-02-11 09:38:36   

              End Time  
0  2022-02-11 09:39:13  


### Affordability Step 2) RAG Calculation
- Take minimum of the SOLETURN across the 3 metrics (3M, 6M 12M)
- Calculate Payments increase based on X Methodology
- Flag To Check If Interest Was Charged In Month (Summary)

In [28]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".affordability_calc_a""")

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".affordability_calc_a as 
select distinct
a.*
-- Take the lowest of the metrics?
,case when (cast(Max_RAG_SOLETURN_3M as int) <= cast(Max_RAG_SOLETURN_6M as int) and cast(Max_RAG_SOLETURN_3M as int) <= cast(Max_RAG_SOLETURN_12M as int)) then Max_RAG_SOLETURN_3M
      when (cast(Max_RAG_SOLETURN_6M as int) <= cast(Max_RAG_SOLETURN_3M as int) and cast(Max_RAG_SOLETURN_6M as int) <= cast(Max_RAG_SOLETURN_12M as int) )then Max_RAG_SOLETURN_6M 
      else Max_RAG_SOLETURN_12M 
 END AS Max_RAG_Soleturn_Avgs
,b.MAX_RAG_BEST_VIEW
,b.ICF_BESTVIEW

,B.AFFORD_SCORE1

,cast(b.Max_RAG_SOLETURN_3M as int) as Max_RAG_SOLETURN_3M
,cast(b.Max_RAG_SOLETURN_6m as int) as Max_RAG_SOLETURN_6m
,cast(b.Max_RAG_SOLETURN_12m as int) as Max_RAG_SOLETURN_12m

,2.5*((power((Retail_apr/100), (1.0/12.0))+0.423)*account_limit - (power((Retail_apr/100), (1.0/12.0))*account_balance)) as payments_increase
  
from "commercial_s3".cobrands.Arun.RBP."{FolderName}".current_apr  a

LEFT JOIN  "commercial_s3".cobrands.Arun.RBP."{FolderName}".affordability b
on a.account_number = b.account_number

""", show_metrics = True);


   Rows Written  Bytes Written Run Time           Start Time  \
0       1711471      111526551  0:00:09  2022-02-11 09:39:20   

              End Time  
0  2022-02-11 09:39:29  


### Affordability Step 3) CATO Calculations
• Current Acccount Turnover categorised into 4 categories based on BESTVIEW and RAG VALUES 


In [29]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".affordability_calc_b""")

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".affordability_calc_b
as
select 
*
        ,case 
            when icf_bestview in (9,8,7,6) and MAX_RAG_BEST_VIEW >= 0 then MAX_RAG_BEST_VIEW -- confident cato available
            when icf_bestview in (3) and MAX_RAG_BEST_VIEW >= 0 then MAX_RAG_BEST_VIEW --decreasing cato available
            when icf_bestview in (3) and MAX_RAG_BEST_VIEW <  0  then 0 --decreasing cato fail
            when icf_bestview is not null and Max_RAG_Soleturn_Avgs  >= 0 then Max_RAG_Soleturn_Avgs --inconsistent cato available
            else 0 
        end as CATO_disposable_income

from "commercial_s3".cobrands.Arun.RBP."{FolderName}".affordability_calc_a 


""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0       1711471      113740036  0:00:04  2022-02-11 09:39:36   

              End Time  
0  2022-02-11 09:39:40  


### Affordability Step 4) Final Calculation Of Pass Fail
• 4 PASS Criteria Else FAIL

In [30]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".affordability_final""")

[[True,
  'Table [commercial_s3.cobrands.Arun.RBP."Debenhams September Backdate".affordability_final] not found.']]

In [31]:
dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".affordability_final
as select 
*

--,case when ICF_BESTVIEW in (9,8,7,6) and MAX_RAG_OTB>= 0 THEN 'PASS1'
--WHEN ICF_BESTVIEW = 3 AND MAX_RAG_BEST_VIEW >= 0 THEN 'PASS2'
--WHEN MAX_RAG_SOLETURN_OTB>=0 THEN 'PASS3'
--WHEN AFFORD_SCORE1 >=240 THEN 'PASS4'
--ELSE 'FAIL'
--END AS AFFORDABILITY_CHECK

,case when CATO_disposable_income >= payments_increase then 'PASS1'
    WHEN AFFORD_SCORE1 >=240 THEN 'PASS2'
    ELSE 'FAIL'
    END AS AFFORDABILITY_CHECK


from "commercial_s3".cobrands.Arun.RBP."{FolderName}".affordability_calc_b

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0       1711471      114784348  0:00:04  2022-02-11 09:40:05   

              End Time  
0  2022-02-11 09:40:09  


### Payment Holiday Data Population

• Find the maximum PH_END_DATE from daily snapshot table per account where ph_flag = 1

In [32]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".PH""")

[[True,
  'Table [commercial_s3.cobrands.Arun.RBP."Debenhams September Backdate".PH] not found.']]

In [33]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".PH""")

dremio_access.execute_sql_ctas(
f"""CREATE TABLE "commercial_s3".cobrands.Arun.RBP."{FolderName}".PH AS

SELECT 
ACCOUNT_NUMBER
,max(ph_end_date) as max_ph_end_date

FROM edm.dm_cb_account_daily_snapshot

where ph_flag = 1
AND RECORD_DATE >='2020-01-01'
group by 1

""", show_metrics = True);


   Rows Written  Bytes Written Run Time           Start Time  \
0         49321         998330  0:00:15  2022-02-11 09:40:28   

              End Time  
0  2022-02-11 09:40:43  


### Payment Freeze Step 1) Extracting Data From Columns

• PF Start and End Data exist in MISC_USER_6 on daily snapshot (example pattern: 20127-20219: YY[DDD] YY[DDD], where DDD = days from start of year)

• Pull distinct entries and extract the Start Year, Start Days, End Year and End Days

In [34]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".pf""")

dremio_access.execute_sql_ctas(
f"""CREATE TABLE "commercial_s3".cobrands.Arun.RBP."{FolderName}".pf AS

SELECT distinct
  account_number
  ,miscellaneous_user_6
  ,CONCAT(CAST(2000+ CAST(substring(miscellaneous_user_6,1,2) AS INTEGER) AS VARCHAR(4)),'-01-01') as pf_start_year
  ,CAST(substring(miscellaneous_user_6,3,3) AS INTEGER)-1 as pf_start_days
  ,CONCAT(CAST(2000+ CAST(substring(miscellaneous_user_6,7,2) AS INTEGER) AS VARCHAR(4)),'-01-01') as pf_end_year
  ,CAST(substring(miscellaneous_user_6,9,3) AS INTEGER)-1 as pf_end_days
  ,case when payment_skip_flag = 'C' then 1 else 0 end as pf_Flag
  FROM edm.dm_cb_account_daily_snapshot
  
  
  where record_date >= '2020-01-01' 
    and payment_skip_flag = 'C'
    AND miscellaneous_user_6 IS NOT NULL
    AND substring(miscellaneous_user_6,1,2) NOT IN ('XX')

""", show_metrics = True)

   Rows Written  Bytes Written Run Time           Start Time  \
0         31765        1603225  0:00:24  2022-02-11 09:41:35   

              End Time  
0  2022-02-11 09:41:59  


### Payment Freeze Step 2) Extracting Data From Columns

• Use previously extracted columns to calculate the actual start and end dates


In [35]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".pf_DATES""")

dremio_access.execute_sql_ctas(
f"""CREATE TABLE "commercial_s3".cobrands.Arun.RBP."{FolderName}".pf_DATES AS

SELECT 
  account_number
--  ,DATE_ADD( to_date(pf_start_year, 'YYYY-MM-DD'),pf_start_days) AS PF_START
  ,max(DATE_ADD( to_date(pf_end_year, 'YYYY-MM-DD'),pf_end_days ) ) AS PF_END
--  , pf_Flag
  
  FROM "commercial_s3".cobrands.Arun.RBP."{FolderName}".pf

group by 1
""", show_metrics = True)

   Rows Written  Bytes Written Run Time           Start Time  \
0         31194         363065  0:00:03  2022-02-11 09:42:14   

              End Time  
0  2022-02-11 09:42:17  


### Payment Freeze/Holiday Final Data Creation

• Use previously extracted columns to calculate the actual start and end dates


In [44]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_pf_ph""")

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_pf_ph
as select 
a.*
,case when PF_END >= '{date_param.This_Month_Plus_6MBack}' then 1
        when max_ph_end_date >= '{date_param.This_Month_Plus_6MBack}' then 1
        else 0 end as pf_ph_l6m


from  "commercial_s3".cobrands.Arun.RBP."{FolderName}".affordability_final a

left join "commercial_s3".cobrands.Arun.RBP."{FolderName}".PH  b
on a.account_number = b.account_number

left join"commercial_s3".cobrands.Arun.RBP."{FolderName}".pf_DATES  c 
on a.account_number = c.account_number


""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0       1711471      115159478  0:00:08  2022-02-11 10:00:38   

              End Time  
0  2022-02-11 10:00:46  


### New Delinquency Last 3M Exclusion

- Track delinqeuncy in past 3 months from summary tables?

In [45]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_dq""")

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_dq
as select 
a.record_date
,a.account_number
,max(delinquent_cycle_period_end) as max_dq_l3m

from  "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_pf_ph a

inner join edm."dm_cb_account_monthly_summary" b
on a.account_number = b.account_number
and b.month_start_date >= '{date_param.This_Month_Plus_3MBack}'

group by 1,2

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0       1711471       17783863  0:00:11  2022-02-11 10:04:57   

              End Time  
0  2022-02-11 10:05:08  


In [46]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_dq_exclusion""")

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_dq_exclusion
as select 
a.*
,b.max_dq_l3m
,case when b.max_dq_l3m >=2 then 1 else 0 end as max_del_l3m_flag
--,case when b.max_dq_l3m >=3 then 1 else 0 end as max_del_l3m_flag_3plus -- not used
from  "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_pf_ph a

inner join  "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_dq b
on a.account_number = b.account_number
AND a.record_date = b.record_date

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0       1711471      117707865  0:00:06  2022-02-11 10:05:19   

              End Time  
0  2022-02-11 10:05:25  


### Paul Jones NonMon Flag Step 1) Track Cycles

In [58]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".InterimCycleJoin """)

dremio_access.execute_sql_ctas(
f"""
create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".InterimCycleJoin
as select 
a.*
,d.record_date as Cycle_Date
FROM "commercial_s3".cobrands.Arun.RBP."Debenhams September Backdate".rbp_dq_exclusion a
LEFT JOIN (SELECT  account_number 
                    ,max(record_date) as record_date
            FROM edm.dm_cb_account_cycle_end_summary d
            WHERE d.record_year_month < {date_param.this_month}
            AND d.record_year_month > 202101
            GROUP BY account_number
                    ) d ON d.account_number = a.account_number

""", show_metrics = True);


   Rows Written  Bytes Written Run Time           Start Time  \
0       1711471      114163004  0:00:03  2022-02-11 10:39:37   

              End Time  
0  2022-02-11 10:39:40  


### Paul Jones NonMon Flag Step 2) Final Join

In [61]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_nonmon_exclusion """)

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_nonmon_exclusion
as select distinct
a.*
,b.ti_us_pr_exclusion_ind as PJ_nonmon_exclusion

from  "commercial_s3".cobrands.Arun.RBP."{FolderName}".InterimCycleJoin a

LEFT JOIN  edl."dl_cb_fd_triad_linkage_uf" b
on a.account_number = b.rr_li_customer_id_min
and a.Cycle_Date = b.record_date

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0       1711471      128490844  0:00:10  2022-02-11 10:42:54   

              End Time  
0  2022-02-11 10:43:04  


### Amazon Exclusion Rules: Bank Will Only Reprice If In The Last 12 Months:
- 1) Missed Payment (Delqineunt)
- 2) Overlimit (Overlimit Fee)
- 3) Breach Of Cardholder Agreement (Other Fees) 

### 1) Missed Payments

In [73]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_dqL12M""")

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_dqL12M
as select 
a.account_number
,a.record_date
,max(delinquent_cycle_period_end) as max_dq_l12m
from "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_nonmon_exclusion a
inner join edm."dm_cb_account_monthly_summary" b
on a.account_number = b.account_number
and b.month_start_date >= '{date_param.This_Month_12MBack_Start}'
group by 1,2

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0       1711471       17996600  0:00:05  2022-02-11 10:51:16   

              End Time  
0  2022-02-11 10:51:21  


In [74]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_dqL12M_exclusion""")

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_dqL12M_exclusion
as select 
a.*
,b.max_dq_l12m
,case when b.max_dq_l12m >=2 then 1 else 0 end as Amazon_Max_Del_L12m_flag
from   "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_nonmon_exclusion a

inner join  "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_dqL12M b
on a.account_number = b.account_number
and a.record_date = b.record_date

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0       1711471      124134175  0:00:05  2022-02-11 10:51:32   

              End Time  
0  2022-02-11 10:51:37  


### 2/3) Overlimit and Late Fees 

In [75]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_OLandLPFeesL12M""")

dremio_access.execute_sql_ctas(
f""" create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_OLandLPFeesL12M  as

SELECT a.account_number
    ,SUM(transaction_amount) as Fees_L12M
    ,sum(CASE WHEN b.gl_code_opp_description = 'Fee Income - Late Fees (Gross)' THEN transaction_amount ELSE NULL END) as Late_Fees_L12M
    ,sum(CASE WHEN b.gl_code_opp_description = 'Fee Income - Overlimit Fees (Gross)' THEN transaction_amount ELSE NULL END) as Overlimit_Fees_L12M
    ,sum(CASE WHEN b.gl_code_opp_description = 'Fee Income - Other Charges' THEN transaction_amount ELSE NULL END) as Other_Fees_L12M
FROM  edm.dm_cb_transaction_fact a

INNER join edm.dr_cb_gl_mapping_dim b
ON  a.transaction_post_type_code = b.transaction_post_type_code

WHERE 1=1 
AND effective_date >=  '{date_param.This_Month_12MBack_Start}'
and transaction_cr_dr_indicator = 'DR'
AND  b.gl_code_opp_description IN ('Fee Income - Late Fees (Gross)'
                                  ,'Fee Income - Other Charges'
                                  ,'Fee Income - Overlimit Fees (Gross)'
                                  )
AND a.account_number IN (SELECT distinct account_number FROM "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_dqL12M_exclusion)

GROUP BY 1

""", show_metrics = True)

   Rows Written  Bytes Written Run Time           Start Time  \
0        343624        5846767  0:00:27  2022-02-11 10:52:39   

              End Time  
0  2022-02-11 10:53:06  


In [76]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_OLandLPFeesL12M_Exclusion""")

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_OLandLPFeesL12M_Exclusion
as select 
a.*
,b.Fees_L12M
,b.Late_Fees_L12M
,b.Overlimit_Fees_L12M
,b.Other_Fees_L12M
,case when b.Fees_L12M >=0 THEN 1 ELSE 0 END as Fees_L12m_flag
,case when b.Late_Fees_L12M >=0 THEN 1 ELSE 0 END as LC_Fees_L12m_flag
,case when b.Overlimit_Fees_L12M >=0 THEN 1 ELSE 0 END as OL_Fees_L12m_flag
,case when b.Other_Fees_L12M >=0 THEN 1 ELSE 0 END as Other_Fees_L12m_flag

from "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_dqL12M_exclusion a

LEFT JOIN "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_OLandLPFeesL12M b
on a.account_number = b.account_number

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0       1711471      129874437  0:00:05  2022-02-11 10:53:20   

              End Time  
0  2022-02-11 10:53:25  


In [77]:
        dremio_access.execute_sql(f""" DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".Amazon_Waterfall_IBB_MarFlag_OrgBrand """);
        
        dremio_access.execute_sql_ctas(f""" CREATE TABLE  "commercial_s3".cobrands.Arun.RBP."{FolderName}".Amazon_Waterfall_IBB_MarFlag_OrgBrand as
                                SELECT * ,CASE WHEN original_brand_raw <> Amazon_Brand Then original_brand_raw ELSE Amazon_Brand END as Original_Brand
                                         ,CASE WHEN opened_date > '2019-03-01' THEN 1 ELSE 0 END AS PostMar2019_Flag
                                         ,CASE WHEN AFFORDABILITY_CHECK IN ('PASS1','PASS2') THEN 1 ELSE 0 END AS AffordabilityPass_Flag
                                         ,CASE WHEN CASE WHEN original_brand_raw <> Amazon_Brand Then original_brand_raw ELSE Amazon_Brand END 
                                            <> Amazon_Brand THEN 1 ELSE 0 END AS Upgrade_Flag
                                FROM 
                                (SELECT a.*
                                ,transfer_from_account_number
                                ,CASE WHEN substring(transfer_from_account_number,4,3) = '736' AND substring(transfer_from_account_number,7,3) = 501 THEN 'Amazon Platinum'
                                      WHEN substring(transfer_from_account_number,4,3)  = '736' AND substring(transfer_from_account_number,7,3) = 511 THEN 'Amazon Classic'
                                END as Original_Brand_Raw
                                FROM "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_OLandLPFeesL12M_Exclusion a
                                INNER JOIN edm.dm_cb_account_cycle_end_summary b 
                                    ON a.account_number = b.account_number
                                   AND a.record_date = b. record_date
                                LEFT JOIN edm.dm_cb_roa_accounts_dim_view c 
                                    ON b.universal_account_identifier = c.universal_account_identifier
                                   AND a.account_number = c.account_number
                                   AND transfer_from_account_number IS NOT NULL
                                ) a
                                """, show_metrics = True);

ERROR IS: `Execution failed on sql ' CREATE TABLE  "commercial_s3".cobrands.Arun.RBP."Debenhams September Backdate".Amazon_Waterfall_IBB_MarFlag_OrgBrand as
                        SELECT * ,CASE WHEN original_brand_raw <> Amazon_Brand Then original_brand_raw ELSE Amazon_Brand END as Original_Brand
                                 ,CASE WHEN opened_date > '2019-03-01' THEN 1 ELSE 0 END AS PostMar2019_Flag
                                 ,CASE WHEN AFFORDABILITY_CHECK IN ('PASS1','PASS2') THEN 1 ELSE 0 END AS AffordabilityPass_Flag
                                 ,CASE WHEN CASE WHEN original_brand_raw <> Amazon_Brand Then original_brand_raw ELSE Amazon_Brand END 
                                    <> Amazon_Brand THEN 1 ELSE 0 END AS Upgrade_Flag
                        FROM 
                        (SELECT a.*
                        ,transfer_from_account_number
                        ,CASE WHEN substring(transfer_from_account_number,4,3) = '736' AND substring(transfer_from_accou

DatabaseError: Execution failed on sql ' CREATE TABLE  "commercial_s3".cobrands.Arun.RBP."Debenhams September Backdate".Amazon_Waterfall_IBB_MarFlag_OrgBrand as
                        SELECT * ,CASE WHEN original_brand_raw <> Amazon_Brand Then original_brand_raw ELSE Amazon_Brand END as Original_Brand
                                 ,CASE WHEN opened_date > '2019-03-01' THEN 1 ELSE 0 END AS PostMar2019_Flag
                                 ,CASE WHEN AFFORDABILITY_CHECK IN ('PASS1','PASS2') THEN 1 ELSE 0 END AS AffordabilityPass_Flag
                                 ,CASE WHEN CASE WHEN original_brand_raw <> Amazon_Brand Then original_brand_raw ELSE Amazon_Brand END 
                                    <> Amazon_Brand THEN 1 ELSE 0 END AS Upgrade_Flag
                        FROM 
                        (SELECT a.*
                        ,transfer_from_account_number
                        ,CASE WHEN substring(transfer_from_account_number,4,3) = '736' AND substring(transfer_from_a

## Create Waterfall Of Exclusion Reasons

In [83]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".Amazon_Waterfall """)

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".Amazon_Waterfall 
as
select
record_date
,account_number
,Post_Mar2019Flag
,opened_date
.CASE WHEN opened_date > '2019-03-01' THEN 1 ELSE 0 END AS PostMar2019_Flag
,user_score_31
,user_field_44 
,client_brand_description
,brand_description
,class
,post_aug_2015_flag
,marketing_channel_on_app
,pct
,retail_apr
,bt_apr
,cash_apr
,interest_charge_flag
,over_75
,AFFORDABILITY_CHECK
,CASE WHEN AFFORDABILITY_CHECK IN ('PASS1','PASS2') THEN 1 ELSE 0 END AS AffordabilityPass_Flag
,blocked
,blocked_l3m
,del_l3m
,max_del_l3m_flag
,card_expired
,dmc
,external_two_plus
,mortgage_arrears
,ccj_l6m
,searches_3_plus_l1m
,less_12_mob
,months_active
,bt_mt_balance_flag
,cli_l5m
,CLI_offer_l1m
,pd_18_flag
,pd_36_cure_date
,fin_diff_flag
,repriced_l6m
,current_balance
,credit_limit
,case when interest_net > 0 THEN current_balance else 0 end AS ME_IBB_Balance
,case when interest_charge_flag <= 0 THEN current_balance else 0 end AS ME_Non_IBB_Balance
,case when trim(upper(class)) = 'PLCC' and opened_date >'2013-10-01' and TRIM(brand_description)='Arcadia' then 0 else 1 end as plcc_repricable 
-- this is arcadia only
,pf_ph_l6m
,inflight_reprice
,case when user_score_31 = 1 then 1 else 0 end as default_score_flag
,case when cast(substring(account_number,4,3) as int) < 736 
       and cast(substring(account_number,7,3) as int) < 680 then 1
    else 0 end as repriceable_org_logo    
,last_reprice_direction
,PJ_nonmon_exclusion
, Amazon_Brand
, CASE WHEN Amazon_APRCAP = 24.94 THEN 29.94
       WHEN Amazon_APRCAP = 29.94 THEN 34.94
 END AS Amazon_APRCAP_Updated
,Amazon_APRCAP
,CASE WHEN Amazon_Max_Del_L12m_flag = 0 AND LC_Fees_L12m_flag = 0 AND OL_Fees_L12m_flag = 0  AND Other_Fees_L12m_flag = 0 THEN 1 ELSE 0 END AS Amazon_CriteriaNotMet
,Amazon_Max_Del_L12m_flag as Amazon_Max_Del_L12m_flag
,LC_Fees_L12m_flag as Amazon_LC_Fees_L12m_flag
,OL_Fees_L12m_flag as Amazon_OL_Fees_L12m_flag
,Other_Fees_L12m_flag as Amazon_Other_Fees_L12m_flag


,case when BLOCKED = 1 then 'A.BLOCKED'
when BLOCKED_L3M = 1 then 'B.BLOCKED_L3M'
when max_del_l3m_flag = 1 then 'C.Del_L3M 2+ cycle'
when Card_expired = 1  then 'D.Card_expired'
when DMC = 1  then 'E.DMC'
when external_two_plus = 1  then 'F.external_2_plus'
when Mortgage_arrears = 1  then 'G.Mortgage_arrears'
when CCJ_L6M = 1 then 'H.CCJ_L6M'
when less_12_mob = 1  then 'I.Less_12MOB'
when months_active <> 3  then 'J.Not Active 3M' 
when bt_mt_balance_flag = 1 then 'K. BT_MT_Balance'
when  CLI_L5M = 1 then 'L. CLI L5M'
when CLI_offer_l1m = 1 then 'M. CLI offer L1M'
when REPRICED_L6M = 1 then 'N. repriced l6m'
when inflight_reprice = 1 then 'O. inflight rbp'
when pd_18_flag = 1 then 'P. pd_18'
when pd_36_cure_date >= '2021-01-01' then 'Q. PD cure < 90 days'
when fin_diff_flag = 1 then 'R.fin diff'
--when AFFORDABILITY_CHECK = 'FAIL' then 'S. affordability fail'
when pf_ph_l6m = 1 then 'T. PF/PH L6M'
when user_score_31 = 1 then 'U. score = 1'
when PJ_nonmon_exclusion = 1 then 'V. PJ nonmon exclusion'
when cast(substring(account_number,7,3) as int) >= 680 then 'W. no strategy'
ELSE 'X.Eligible'
end as original_waterfall

,case when BLOCKED = 1 then 'A.BLOCKED'
when BLOCKED_L3M = 1 then 'B.BLOCKED_L3M'
when max_del_l3m_flag = 1 then 'C.Del_L3M 2+ cycle'
when Card_expired = 1  then 'D.Card_expired'
when DMC = 1  then 'E.DMC'
when external_two_plus = 1  then 'F.external_2_plus'
when Mortgage_arrears = 1  then 'G.Mortgage_arrears'
when CCJ_L6M = 1 then 'H.CCJ_L6M'
when less_12_mob = 1  then 'I.Less_12MOB'
when months_active <> 3  then 'J.Not Active 3M' 
when bt_mt_balance_flag = 1 then 'K. BT_MT_Balance'
when  CLI_L5M = 1 then 'L. CLI L5M'
when CLI_offer_l1m = 1 then 'M. CLI offer L1M'
when REPRICED_L6M = 1 then 'N. repriced l6m'
when inflight_reprice = 1 then 'O. inflight rbp'
when pd_18_flag = 1 then 'P. pd_18'
when pd_36_cure_date >= '2021-01-01' then 'Q. PD cure < 90 days'
when fin_diff_flag = 1 then 'R.fin diff'
--when AFFORDABILITY_CHECK = 'FAIL' then 'S. affordability fail'
when pf_ph_l6m = 1 then 'T. PF/PH L6M'
when user_score_31 = 1 then 'U. score = 1'
when PJ_nonmon_exclusion = 1 then 'V. PJ nonmon exclusion'
WHEN Amazon_Max_Del_L12m_flag = 0 AND LC_Fees_L12m_flag = 0 AND OL_Fees_L12m_flag = 0  AND Other_Fees_L12m_flag = 0 THEN 'W.Amazon_CriteriaNoMet'
when cast(substring(account_number,7,3) as int) >= 680 then 'X. no strategy'
ELSE 'Y.Eligible'
end as Amazon_Waterfall

from "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_OLandLPFeesL12M_Exclusion
WHERE cast(substring(account_number,4,3) as int) = 730

""", show_metrics = True);

ERROR IS: `Execution failed on sql 'create table "commercial_s3".cobrands.Arun.RBP."Debenhams September Backdate".Amazon_Waterfall 
as
select
record_date
,account_number
,opened_date
,CASE WHEN opened_date > '2019-03-01' THEN 1 ELSE 0 END AS PostMar2019_Flag
,user_score_31
,user_field_44 
,client_brand_description
,brand_description
,class
,post_aug_2015_flag
,marketing_channel_on_app
,pct
,retail_apr
,bt_apr
,cash_apr
,interest_charge_flag
,over_75
,AFFORDABILITY_CHECK
,CASE WHEN AFFORDABILITY_CHECK IN ('PASS1','PASS2') THEN 1 ELSE 0 END AS AffordabilityPass_Flag
,blocked
,blocked_l3m
,del_l3m
,max_del_l3m_flag
,card_expired
,dmc
,external_two_plus
,mortgage_arrears
,ccj_l6m
,searches_3_plus_l1m
,less_12_mob
,months_active
,bt_mt_balance_flag
,cli_l5m
,CLI_offer_l1m
,pd_18_flag
,pd_36_cure_date
,fin_diff_flag
,repriced_l6m
,current_balance
,credit_limit
,case when interest_charge_flag > 0 THEN current_balance else 0 end AS ME_IBB_Balance
,case when interest_charge_flag <= 0 THEN curre

DatabaseError: Execution failed on sql 'create table "commercial_s3".cobrands.Arun.RBP."Debenhams September Backdate".Amazon_Waterfall 
as
select
record_date
,account_number
,opened_date
,CASE WHEN opened_date > '2019-03-01' THEN 1 ELSE 0 END AS PostMar2019_Flag
,user_score_31
,user_field_44 
,client_brand_description
,brand_description
,class
,post_aug_2015_flag
,marketing_channel_on_app
,pct
,retail_apr
,bt_apr
,cash_apr
,interest_charge_flag
,over_75
,AFFORDABILITY_CHECK
,CASE WHEN AFFORDABILITY_CHECK IN ('PASS1','PASS2') THEN 1 ELSE 0 END AS AffordabilityPass_Flag
,blocked
,blocked_l3m
,del_l3m
,max_del_l3m_flag
,card_expired
,dmc
,external_two_plus
,mortgage_arrears
,ccj_l6m
,searches_3_plus_l1m
,less_12_mob
,months_active
,bt_mt_balance_flag
,cli_l5m
,CLI_offer_l1m
,pd_18_flag
,pd_36_cure_date
,fin_diff_flag
,repriced_l6m
,current_balance
,credit_limit
,case when interest_charge_flag > 0 THEN current_balance else 0 end AS ME_IBB_Balance
,case when interest_charge_flag <= 0 THEN c

In [84]:
df = dremio_access.read_sql_to_dataframe(
     f"""
        SELECT *
        FROM "commercial_s3".cobrands.Arun.RBP."{FolderName}".Debenhams_Waterfall
      """
    )



In [85]:
df

,record_date,account_number,opened_date,PostMar2019_Flag,user_score_31,user_field_44,client_brand_description,brand_description,class,post_aug_2015_flag,...,repriceable_org_logo,last_reprice_direction,PJ_nonmon_exclusion,Amazon_CriteriaNotMet,Amazon_Max_Del_L12m_flag,Amazon_LC_Fees_L12m_flag,Amazon_OL_Fees_L12m_flag,Amazon_Other_Fees_L12m_flag,original_waterfall,Amazon_Waterfall
0,2021-08-31,0007306050100737585,2008-03-06,0,0347,347,Debenhams,Debenhams,PLCC,0,...,1,None,1.0,0,0,1,0,0,V. PJ nonmon exclusion,V. PJ nonmon exclusion
1,2021-08-31,0007305010019507686,2019-07-24,1,0341,341,Debenhams,Debenhams,RBCC,1,...,1,UP,1.0,0,0,1,0,0,V. PJ nonmon exclusion,V. PJ nonmon exclusion
2,2021-08-31,0007306000001348495,2005-06-21,0,0393,393,Debenhams,Debenhams,PLCC,0,...,1,None,1.0,1,0,0,0,0,A.BLOCKED,A.BLOCKED
3,2021-08-31,0007305050206650089,2009-02-17,0,0325,325,Debenhams,Debenhams,RBCC,0,...,1,None,NaN,0,0,1,0,0,J.Not Active 3M,J.Not Active 3M
4,2021-08-31,0007305010020955510,2020-01-03,1,0274,274,Debenhams,Debenhams,RBCC,1,...,1,UP,1.0,0,0,1,0,0,V. PJ nonmon exclusion,V. PJ nonmon exclusion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805529,2021-08-31,0007306000100069893,2009-04-24,0,0364,364,Debenhams,Debenhams,PLCC,0,...,1,None,1.0,1,0,0,0,0,A.BLOCKED,A.BLOCKED
805530,2021-08-31,0007306050001524306,1996-02-05,0,0392,392,Debenhams,Debenhams,PLCC,0,...,1,None,1.0,1,0,0,0,0,A.BLOCKED,A.BLOCKED
805531,2021-08-31,0007305010018256848,2019-03-11,1,0387,387,Debenhams,Debenhams,RBCC,1,...,1,None,1.0,1,0,0,0,0,J.Not Active 3M,J.Not Active 3M
805532,2021-08-31,0007305010011805898,2017-07-22,0,0387,387,Debenhams,Debenhams,RBCC,1,...,1,None,1.0,1,0,0,0,0,J.Not Active 3M,J.Not Active 3M


In [86]:
df.to_csv('202201_rbp_DebenhamsWaterfall_Sept.csv', index = True)

## 2021- Oct. The channel lookup table has replaced Laura Ashley with Pulse (LA) and similarly for other pulse brands.
## hence their strategies will automatically not work in my code due to naming differences

## define strategies

In [56]:
dremio_access.execute_sql("""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".eligible_strat """)

[[True,
  'Table [commercial_s3.cobrands.Perry."RBP Sep21 Volume Estimate".eligible_strat] dropped']]

In [90]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".eligible_strat """)

dremio_access.execute_sql_ctas(
f"""create table  "commercial_s3".cobrands.Arun.RBP."{FolderName}".eligible_strat
as  
SELECT
record_date
,user_score_31
,user_field_44
,client_brand_description
,brand_description
,CLASS
,POST_AUG_2015_flag
,MARKETING_CHANNEL_ON_APP
,OVER_75
,ACCOUNT_NUMBER
,PCT
,RETAIL_APR
,BT_APR
,CASH_APR
,AFFORDABILITY_CHECK
,AffordabilityPass_Flag
,current_balance
,interest_charge_flag
,last_reprice_direction

,CASE
WHEN  TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 0 and CLASS = 'RBCC' THEN 'DEBS MC BB'
WHEN TRIM(client_brand_description) = 'Debenhams' and CLASS = 'PLCC' THEN 'DEBS SC'
WHEN TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 1 and CLASS = 'RBCC' and marketing_channel_on_app = 'POS' THEN 'DEBS MC POS POST AUG 2015'
WHEN TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 1 and CLASS = 'RBCC' and marketing_channel_on_app = 'Internet' THEN 'DEBS MC INTERNET POST AUG 2015'
END AS RBP_STRATEGY

,case
WHEN  TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 0 and CLASS = 'RBCC'  AND RETAIL_APR  <=27.9 THEN 24.9
WHEN TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 0 and CLASS = 'RBCC' AND RETAIL_APR  BETWEEN 28.0 AND 31.9 THEN 29.9
WHEN TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 0 and CLASS = 'RBCC' AND RETAIL_APR  <=37.9 THEN 34.9
WHEN TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 0 and CLASS = 'RBCC' AND RETAIL_APR  <=39.9 THEN 39.9

when TRIM(client_brand_description) = 'Debenhams' and CLASS = 'PLCC' AND RETAIL_APR  <=28.1 THEN 26
when TRIM(client_brand_description) = 'Debenhams' and CLASS = 'PLCC' AND RETAIL_APR  <=31.2 THEN 31.2
when TRIM(client_brand_description) = 'Debenhams' and CLASS = 'PLCC' AND RETAIL_APR  <=39.8 THEN 36.4
when TRIM(client_brand_description) = 'Debenhams' and CLASS = 'PLCC' AND RETAIL_APR  <=39.9 THEN 39.9


else cast(RETAIL_APR as decimal(4,2))
end as retail_apr_override

FROM "commercial_s3".cobrands.Arun.RBP."{FolderName}".Debenhams_Waterfall 
WHERE RETAIL_APR > 0
and BLOCKED = 0
and BLOCKED_L3M = 0
and Del_L3M = 0
and Card_expired= 0
and DMC= 0
and external_two_plus= 0
and Mortgage_arrears= 0
and CCJ_L6M= 0
and less_12_mob= 0
and months_active = 3
and bt_mt_balance_flag = 0
and CLI_L5M= 0
and CLI_offer_l1m = 0
and REPRICED_L6M= 0
and (pd_18_flag is null or pd_18_flag= 0)
and fin_diff_flag = 0
and plcc_repricable = 1
and pf_ph_l6m = 0
and inflight_reprice = 0
and default_score_flag = 0
and (case when pd_36_cure_date is null then 1
    when pd_36_cure_date < '2021-07-01' then 1 end) = 1

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0        151656        5762055  0:00:03  2022-02-11 11:38:31   

              End Time  
0  2022-02-11 11:38:34  


## need to limit it to accounts with a strategy 

In [91]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".eligible_strat_2 """)

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".eligible_strat_2
as select 
*
from "commercial_s3".cobrands.Arun.RBP."{FolderName}".eligible_strat
where RBP_STRATEGY IS NOT NULL

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0        151656        5762055  0:00:01  2022-02-11 11:38:50   

              End Time  
0  2022-02-11 11:38:51  


In [61]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".strat_hardcode """)

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".strat_hardcode 
AS
SELECT
A.*

,case when TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 0 and CLASS = 'RBCC' AND user_field_44  <=363 THEN 39.9
when TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 0 and CLASS = 'RBCC' AND user_field_44  <=378 THEN 34.9
when TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 0 and CLASS = 'RBCC' AND user_field_44  <=999 THEN 24.9

--Debs storecard 
when TRIM(client_brand_description) = 'Debenhams' and CLASS = 'PLCC' AND user_field_44  <=250 THEN 39.9
when TRIM(client_brand_description) = 'Debenhams' and CLASS = 'PLCC' AND user_field_44  <=384 THEN 36.4
when TRIM(client_brand_description) = 'Debenhams' and CLASS = 'PLCC' AND user_field_44  <=391 THEN 31.2
when TRIM(client_brand_description) = 'Debenhams' and CLASS = 'PLCC' AND user_field_44  <=999 THEN 26.0

-- Debs Instore MC with 39.9 expansion 
WHEN TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 1 and CLASS = 'RBCC' and marketing_channel_on_app = 'POS' AND user_field_44  <=363 THEN 39.9
WHEN TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 1 and CLASS = 'RBCC' and marketing_channel_on_app = 'POS' AND user_field_44  <=378 THEN 34.9
WHEN TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 1 and CLASS = 'RBCC' and marketing_channel_on_app = 'POS' AND user_field_44  <=999 THEN 24.9

-- Debs INTERNET MC with 39.9 expansion
WHEN TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 1 and CLASS = 'RBCC' and marketing_channel_on_app = 'Internet' AND user_field_44  <=324 THEN 39.9
WHEN TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 1 and CLASS = 'RBCC' and marketing_channel_on_app = 'Internet' AND user_field_44  <=357 THEN 34.9
WHEN TRIM(client_brand_description) = 'Debenhams' AND POST_AUG_2015_flag = 1 and CLASS = 'RBCC' and marketing_channel_on_app = 'Internet' AND user_field_44  <=999 THEN 24.9
END AS TARGET_APR

FROM "commercial_s3".cobrands.Arun.RBP."{FolderName}".eligible_strat_2 A
WHERE user_field_44 <> 100

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0        198965       10077396  0:00:21  2021-10-27 16:25:19   

              End Time  
0  2021-10-27 16:25:40  


In [63]:
dremio_access.execute_sql(f"""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".repr_dir """)

dremio_access.execute_sql_ctas(
f"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".repr_dir
AS
SELECT
a.record_date
,a.user_score_31
,a.user_field_44
,a.client_brand_description
,a.brand_description
,a.CLASS
,a.POST_AUG_2015_flag
,a.MARKETING_CHANNEL_ON_APP
,a.OVER_75
,a.ACCOUNT_NUMBER
,a.PCT
,cast(a.RETAIL_APR as decimal(4,2)) RETAIL_APR
,cast(a.BT_APR as decimal(4,2)) BT_APR
,cast(a.CASH_APR as decimal(4,2)) CASH_APR
,a.AFFORDABILITY_CHECK
,a.current_balance
,a.interest_charge_flag
,a.RBP_STRATEGY
,a.retail_apr_override
,a.TARGET_APR
,a.last_reprice_direction
,CASE when TARGET_APR >= retail_apr_override + 3 AND OVER_75 = 0 and AFFORDABILITY_CHECK <> 'FAIL' and isnull(last_reprice_direction) THEN 'UP'
    WHEN TARGET_APR >= retail_apr_override + 3 AND OVER_75 = 0 and AFFORDABILITY_CHECK <> 'FAIL' and last_reprice_direction = 'UP' THEN 'UP'

    WHEN TARGET_APR <= retail_apr_override - 3 and isnull(last_reprice_direction) THEN 'DOWN'
    WHEN TARGET_APR <= retail_apr_override - 3 and last_reprice_direction = 'DOWN' THEN 'DOWN'
    END AS REPR_DIR
--no opposite direction reprice, need to treat nulls separately


FROM  "commercial_s3".cobrands.Arun.RBP."{FolderName}".strat_hardcode  a


""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0        198965       10346050  0:00:03  2021-10-27 16:25:42   

              End Time  
0  2021-10-27 16:25:45  


In [64]:
dremio_access.execute_sql("""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_summ_dir """)

[[True,
  'Table [commercial_s3.cobrands.Perry."RBP Sep21 Volume Estimate".rbp_summ_dir] dropped']]

In [65]:
dremio_access.execute_sql_ctas(
"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".rbp_summ_dir
AS
SELECT
client_brand_description
,brand_description
,CLASS
,POST_AUG_2015_flag
,MARKETING_CHANNEL_ON_APP
,OVER_75
,PCT
,cast(RETAIL_APR as decimal(4,2)) RETAIL_APR
,cast(BT_APR as decimal(4,2)) BT_APR
,cast(CASH_APR as decimal(4,2)) CASH_APR
,AFFORDABILITY_CHECK
,RBP_STRATEGY
,retail_apr_override as retail_apr_group
,TARGET_APR
,REPR_DIR

,count(account_number) as accts
,sum(current_balance) as current_balance
,sum(case when interest_charge_flag = 1 then current_balance else 0 end) as IBB

FROM  "commercial_s3".cobrands.Arun.RBP."{FolderName}".repr_dir 



group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0          2168         105301  0:00:04  2021-10-27 16:25:46   

              End Time  
0  2021-10-27 16:25:50  


In [66]:
dremio_access.execute_sql("""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".waterfall_reprice_dir """)

[[True,
  'Table [commercial_s3.cobrands.Perry."RBP Sep21 Volume Estimate".waterfall_reprice_dir] dropped']]

In [67]:
dremio_access.execute_sql_ctas(
"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".waterfall_reprice_dir
AS select
a.*

,b.REPR_DIR
,b.RBP_STRATEGY
,b.retail_apr_override
,b.TARGET_APR
,c.strategy_row_translation
,C.trdrfocr_fo_stgy_row

FROM  "commercial_s3".cobrands.Arun.RBP."{FolderName}".waterfall a

left join  "commercial_s3".cobrands.Arun.RBP."{FolderName}".repr_dir b
on a.account_number = b.account_number

left join "commercial_s3".cobrands.Arun.RBP."{FolderName}".estimator_data  c
on a.account_number = c.trdrfocr_hdr_customer_id

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0       1043232       59938511  0:00:03  2021-10-27 16:25:52   

              End Time  
0  2021-10-27 16:25:55  


In [68]:
dremio_access.execute_sql("""DROP TABLE IF EXISTS "commercial_s3".cobrands.Arun.RBP."{FolderName}".estimator_nexus_comparison """)

[[True,
  'Table [commercial_s3.cobrands.Perry."RBP Sep21 Volume Estimate".estimator_nexus_comparison] dropped']]

In [69]:
dremio_access.execute_sql_ctas(
"""create table "commercial_s3".cobrands.Arun.RBP."{FolderName}".estimator_nexus_comparison as

SELECT
client_brand_description
,brand_description
,CLASS
,POST_AUG_2015_flag
,MARKETING_CHANNEL_ON_APP
,OVER_75
,PCT
,cast(RETAIL_APR as decimal(4,2)) RETAIL_APR
,cast(BT_APR as decimal(4,2)) BT_APR
,cast(CASH_APR as decimal(4,2)) CASH_APR
,AFFORDABILITY_CHECK
,RBP_STRATEGY
,retail_apr_override as retail_apr_group
,TARGET_APR
, REPR_DIR
,strategy_row_translation
,user_score_31
,count(account_number) as accts
,sum(current_balance) as current_balance
,sum(case when interest_charge_flag = 1 then current_balance else 0 end) as IBB

FROM "commercial_s3".cobrands.Arun.RBP."{FolderName}".waterfall_reprice_dir 

where REPR_DIR = 'UP'
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17

""", show_metrics = True);

   Rows Written  Bytes Written Run Time           Start Time  \
0          3164         170613  0:00:03  2021-10-27 16:25:56   

              End Time  
0  2021-10-27 16:25:59  
